### Добавление, проверка и извлечение всплесков из JSON

In [4]:
import numpy as np

from classic.wave import wavedec, waverec, convolve
from metrics import psnr
from offset_tensor import OffsetTensor
from periodic.wave import wavedec_period, waverec_period
from wavelet import Wavelet

from utils import OffsetMatrixConjugate

##### Код

In [4]:
# Из Git
def to_python(x, y, offset):
    return x - offset[0], y - offset[1]


def to_coord(row, col, offset):
    return offset[0]+col, offset[1]-row


def to_python_vect(coords, offset):
    return coords.T[0] - offset[0], coords.T[1]-offset[1]

def CoefCoords2OffsetMatrix(a):  # a filter in coef-coords form
    coef = a[0]
    coords = a[1]
    x_left, y_bottom = list(np.min(np.transpose(coords), axis=1))
    x_right, y_up = list(np.max(np.transpose(coords), axis=1))
    shape = [x_right-x_left+1, y_up-y_bottom+1]
    offset = np.array([x_left, y_bottom])
    matrix = np.zeros(shape)
    for ind, coord in enumerate(coords):
        row, col = to_python(coord[0], coord[1], offset)
        matrix[row, col] = coef[ind]
    return OffsetMatrix(matrix, offset)

def OffsetMatrix2CoefCoords(a: OffsetMatrix):
    shape = a.matrix.shape
    offset = a.offset
    coef = []
    coords = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            if a.matrix[i, j] != 0:
                coef.append(a.matrix[i, j])
                coords.append([offset[0]+i, offset[1]+j])
    if len(coef) == 0:
        coef, coords = [0], [[0,0]]
    return np.array(coef), np.array(coords)

In [5]:
from itertools import product as direct_product

def createWaveletFromContent(content):
    M = np.array(content['DilationMatrix'])
    coeffs = np.array(content['Mask']['Coeffs']) / content['Mask']['CoeffsDenominator']
    coords = np.array(content['Mask']['Coords'])
    h = CoefCoords2OffsetMatrix((coeffs,coords))

    coeffs = np.array(content['DualMask']['Coeffs']) / content['DualMask']['CoeffsDenominator']
    coords = np.array(content['DualMask']['Coords'])
    hdual = CoefCoords2OffsetMatrix((coeffs,coords))
    
    g = []
    for wmask in content['WaveletMasks']:
        coeffs = np.array(content['WaveletMasks'][wmask]['Coeffs']) / content['WaveletMasks'][wmask]['CoeffsDenominator']
        coords = np.array(content['WaveletMasks'][wmask]['Coords'])
        g.append(CoefCoords2OffsetMatrix((coeffs,coords)))
    g = tuple(g)

    gdual = []
    for wmask in content['DualWaveletMasks']:
        coeffs = np.array(content['DualWaveletMasks'][wmask]['Coeffs']) / content['DualWaveletMasks'][wmask]['CoeffsDenominator']
        coords = np.array(content['DualWaveletMasks'][wmask]['Coords'])
        gdual.append(CoefCoords2OffsetMatrix((coeffs,coords)))
    gdual = tuple(gdual)   

    return Wavelet(h, g, hdual, gdual, M, abs(np.linalg.det(M)))





def DeltaN(n, d):
    cube = direct_product(range(n), repeat=d)
    result = []
    for coord in cube:
        if sum(coord) < n:
            result.append(list(coord))
    return result

def PlaneN(n, d):
    cube = direct_product(range(n), repeat=d)
    result = []
    for coord in cube:
        if sum(coord) == n-1:
            result.append(list(coord))
    return result

def MatrixOfPowers(aCoords, bPowers):
    matrix = np.zeros((len(aCoords), len(bPowers)))
    for i, ai in enumerate(aCoords):
        for j, bj in enumerate(bPowers):
            matrix[i,j] = np.prod(np.power(ai, bj))
    return matrix

def VM(wmask: OffsetMatrix, tolerance=1e-6, max_vm=None):
    """
    Calculate the order of vanishing moments a wavelet filter.
    
    Args:
        wmask (OffsetMatrix): Wavelet filter in the OffsetMatrix form.
        tolerance (float, optional): Numerical tolerance to consider sum as zero. Defaults to 1e-6.
        max_vm (int, optional): Maximum order to check. Defaults to the length of coefficients.
    
    Returns:
        int: Order of vanishing moments.
    """
    (coefs, coords) = OffsetMatrix2CoefCoords(wmask)
    d = len(wmask.offset)
    vm = 1
    if max_vm is None:
        max_vm = len(coefs)
    
    while vm <= max_vm:
        bPowers = PlaneN(vm, d)
        MatrixOfPowers(coords, bPowers)
        vm_check = np.dot(coefs, MatrixOfPowers(coords, bPowers))

        if sum(abs(vm_check)) < tolerance:
            vm += 1
        else:
            break
    
    return vm - 1

def SetOfDigitsFinder(M):
    dim = len(M)
    Minv = np.linalg.inv(M)

    min_corner = np.zeros(dim, dtype = np.int32)
    max_corner = np.ones(dim, dtype = np.int32)
    cube_corners = M @ np.array(list(direct_product(*zip(list(min_corner), list(max_corner))))).T

    all_min_corner = np.ceil(np.min(cube_corners, axis = 1)).astype(np.int32)
    all_max_corner = np.floor(np.max(cube_corners, axis = 1)).astype(np.int32)
    cube_inside_coords = np.mgrid[*tuple(map(slice, tuple(all_min_corner), tuple(all_max_corner+np.ones(dim, dtype = np.int32))))].reshape(dim, -1)  

    Mcoords = (Minv @ cube_inside_coords).T
    mask = np.all(np.floor(Mcoords).astype(np.int32) == np.zeros(dim, dtype = np.int32), axis=1)
    return cube_inside_coords.T[mask]

def Mask2Polyphase(mask: OffsetMatrix, M, digits = None):
    if digits is None:
        digits = SetOfDigitsFinder(M)
        
    (coefs, coords) = OffsetMatrix2CoefCoords(mask)
    m = round(np.abs(np.linalg.det(M)))
    Minv = np.linalg.inv(M)
    Minv_pre = np.rint((m * np.linalg.inv(M))).astype(np.int32)
    dim = len(M)

    polyphases = []
    for digit in digits:
        pre_poly_coords = Minv_pre @ (coords - digit).T
        mask = np.all(np.mod(pre_poly_coords, m) == 0, axis=0)
        poly_coords = pre_poly_coords.T[mask]//m
        poly_coefs = coefs[mask]
        if len(poly_coefs)==0:
            polyphases.append(CoefCoords2OffsetMatrix(([0], [np.zeros(dim, dtype = np.int32)])))
        else:
            polyphases.append(CoefCoords2OffsetMatrix((poly_coefs, poly_coords)))

    return polyphases

def SR(mask: OffsetMatrix, M, digits = None, tolerance=1e-6, max_sr=None):
    """
    Calculate the order of sum rule of filter.
    
    Args:
        mask (OffsetMatrix): Wavelet filter in the OffsetMatrix form.
        M: matrix dilation.
        digits: the set of digits
        tolerance (float, optional): Numerical tolerance to consider sum as zero. Defaults to 1e-6.
        max_sr (int, optional): Maximum order to check. Defaults to the length of coefficients.
    
    Returns:
        int: Order of sum rule.
    """
    if digits is None:
        digits = SetOfDigitsFinder(M)
    polyphases = Mask2Polyphase(mask, M)
    d = len(M)
    sr = 1
    if max_sr is None:
        max_sr = 20
    
    while sr <= max_sr:
        bPowers = PlaneN(sr, d)
        sr_check = None
        sr_stop = False
        for digit, polyphase in zip(digits, polyphases):
            (poly_coefs, poly_coords) = OffsetMatrix2CoefCoords(polyphase)
            poly_coords = (poly_coords @ M.T) + digit
            if sr_check is None:
                sr_check = np.dot(poly_coefs, MatrixOfPowers(poly_coords, bPowers))
            else:
                if not np.allclose(sr_check, np.dot(poly_coefs, MatrixOfPowers(poly_coords, bPowers)), atol = tolerance):
                    sr_stop = True
        if sr_stop:
            break
        sr = sr + 1
        if sr > max_sr:
            break      

    return sr - 1

def PRP_check(w: Wavelet, digits = None, tolerance = 1e-6):
    if digits is None:
        digits = SetOfDigitsFinder(w.M)
    polyphase_matrix = []
    dual_polyphase_matrix = []
    dim = len(w.M)
    polyphase_matrix.append(Mask2Polyphase(w.h, w.M, digits))
    dual_polyphase_matrix.append(Mask2Polyphase(w.hdual, w.M, digits))
    for wmask in w.g:
        polyphase_matrix.append(Mask2Polyphase(wmask, w.M, digits))
    for dwmask in w.gdual:
        dual_polyphase_matrix.append(Mask2Polyphase(dwmask, w.M, digits))    

    PRP_matrix = []
    PRP_valid = True
    m = int(w.m)
    for i in range(m):
        PRP_matrix.append([])
        for j in range(m):
            tmp = OffsetMatrix(np.array([[0]]), np.array(np.zeros(dim, dtype = np.int32)))
            for k in range(len(w.g)+1):
                tmp = tmp + convolve(polyphase_matrix[k][i], OffsetMatrixConjugate(dual_polyphase_matrix[k][j]))
            
            if i == j:
                tmp_check = np.zeros_like(tmp.matrix)
                tmp_check[*(-np.array(tmp.offset))] = 1/w.m
                if abs(np.sum(np.abs(tmp.matrix-tmp_check))) > tolerance:
                    PRP_valid = False
                    print("False:", i,' ', j, ' ', tmp.matrix)
            else:
                if abs(np.sum(np.abs(tmp.matrix))) > tolerance:
                    PRP_valid = False
                    print("False:", i,' ', j, ' ', tmp.matrix)
                    
            PRP_matrix[i].append(tmp)
    return PRP_valid, PRP_matrix

def checkContent(content):
    w = createWaveletfromContent(content)
    if SR(w.h, w.M) == content['Mask']['SR']:
        print("Mask SR OK")
    else:
        print("Mask SR wrong: by checker SR = ", SR(w.h, w.M),
             ", by Database SR = ", content['Mask']['SR'])
    if SR(w.hdual, w.M) == content['DualMask']['SR']:
        print("Dual Mask SR OK")
    else:
        print("Dual Mask SR wrong: by checker SR = ", SR(w.h, w.M),
             ", by Database SR = ", content['DualMask']['SR'])

    for wmask in w.g:
        if VM(wmask) >= content['WaveletMasksVM']:
            print("Wavelet Mask VM OK")
        else:
            print("Wavelet Mask VM wrong: by checker VM = ", VM(wmask),
                 ", by Database VM = ", content['WaveletMasksVM'])
        
    for wmask in w.gdual:
        if VM(wmask) >= content['DualWaveletMasksVM']:
            print("Dual Wavelet Mask VM OK")
        else:
            print("Dual Wavelet Mask VM wrong: by checker VM = ", VM(wmask),
                 ", by Database VM = ", content['DualWaveletMasksVM'])
    PRP_valid, _ = PRP_check(w)
    if PRP_valid:
        print("PRP check OK")
    else:
        print("PRP check Wrong")

NameError: name 'OffsetMatrix' is not defined

##### Проверка работы

In [1]:
import json
with open("WaveDB.json", 'r') as j:
     contents = json.loads(j.read())

In [2]:
len(contents)

12

In [3]:
for content in contents:
    print(content['DilationMatrixInfo'])
    print(content['WaveletSystemType'])
    print(content['InterpolatoryMask'])
    print(content['Mask']['Coeffs'])
    print(content['Mask']['SobolevSmoothness'])
    print(content['DualMask']['Coeffs'])
    print(content['DualMask']['SobolevSmoothness'])

QuincunxRotation
Frame-like Dual Wavelet System
True
[2, 1, 1]
1.09619
[1]
-inf
QuincunxRotation
Frame-like Dual Wavelet System in L2
True
[2, 1, 1]
1.09619
[1, 2, 1]
1.09619
QuincunxRotation
Frame-like Dual Wavelet System
True
[2, 1, 1]
1.09619
[8, 2, 2, -1, -1, -1, -1]
-0.932076
QuincunxRotation
Frame-like Dual Wavelet System
True
[4, 1, 1, 1, 1]
1.57764
[1]
-inf
QuincunxRotation
Frame-like Dual Wavelet System in L2
True
[4, 1, 1, 1, 1]
1.57764
[2, 1, 1]
1.09619
QuincunxRotation
Dual Wavelet Frame
True
[4, 1, 1, 1, 1]
1.57764
[1272, 288, 288, 240, 240, -114, -114, -114, -114, -52, -52, -44, -44, -36, -36, -36, -36, -28, -28, -44, -44, 18, 18, 18, 18, 9, 9, 9, 9, 7, 7, 11, 11]
0.300855


In [44]:
content['DilationMatrix']

[[1, -1], [1, 1]]

In [56]:
wnew = createWaveletFromContent(contents[5])
wnew

Wavelet(h=<offset_matrix.OffsetMatrix object at 0x0000020E6CC4A210>, g=(<offset_matrix.OffsetMatrix object at 0x0000020E6CC4A450>,), hdual=<offset_matrix.OffsetMatrix object at 0x0000020E6CC4A330>, gdual=(<offset_matrix.OffsetMatrix object at 0x0000020E6CC4A570>,), M=array([[ 1, -1],
       [ 1,  1]]), m=np.float64(2.0))

In [57]:
wnew.g[0].matrix

array([[ 0.        ,  0.        ,  0.00585938,  0.        ,  0.00716146,
         0.        ,  0.        ],
       [ 0.        ,  0.01171875,  0.0234375 , -0.03385417,  0.02864583,
         0.01171875,  0.        ],
       [ 0.00585938,  0.0234375 , -0.07421875, -0.1875    , -0.07421875,
         0.01822917,  0.00455729],
       [ 0.        , -0.02864583, -0.15625   ,  0.828125  , -0.15625   ,
        -0.02864583,  0.        ],
       [ 0.00455729,  0.01822917, -0.07421875, -0.1875    , -0.07421875,
         0.0234375 ,  0.00585938],
       [ 0.        ,  0.01171875,  0.02864583, -0.03385417,  0.0234375 ,
         0.01171875,  0.        ],
       [ 0.        ,  0.        ,  0.00716146,  0.        ,  0.00585938,
         0.        ,  0.        ]])

In [29]:
wnew.g[0].offset

array([ 0, -1])

In [40]:
checkContent(contents[4])

IndexError: list index out of range